In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, multilabel_confusion_matrix
from collections import defaultdict

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import math


In [3]:
# -------------------------
# PART 1: 自定義 model 與 loss (PyTorch)
# -------------------------

# Feature extractor: 接受 (batch, timesteps, channels) -> 回傳 (batch, new_timesteps, out_channels)
def build_feature_extractor(input_shape):
    """
    input_shape: (timesteps, channels)
    returns: nn.Module whose forward accepts tensor shaped (B, T, C) and returns (B, T', C_out)
    """
    class FeatureExtractor(nn.Module):
        def __init__(self, in_channels):
            super().__init__()
            # mirror TF Conv1D(filters=128, kernel_size=4, padding='same') + pool
            # Use padding = (kernel_size - 1) // 2 to approximate 'same' for stride=1
            self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=128, kernel_size=4, stride=1, padding=1)
            self.pool_avg = nn.AvgPool1d(kernel_size=4)
            self.conv2 = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=4, stride=1, padding=1)
            self.pool_max = nn.MaxPool1d(kernel_size=4)

        def forward(self, x):
            # Expect x shape: (B, T, C). Convert to channels-first for Conv1d
            if x.dim() == 3:
                x = x.permute(0, 2, 1)  # -> (B, C, T)
            elif x.dim() == 2:
                x = x.unsqueeze(1)  # (B,1,T) if input was (B,T)
            # convs
            y = F.relu(self.conv1(x))
            y = self.pool_avg(y)
            y = F.relu(self.conv2(y))
            y = self.pool_max(y)  # (B, C_out, T')
            # back to (B, T', C_out)
            y = y.permute(0, 2, 1)
            return y

    timesteps, channels = input_shape
    return FeatureExtractor(in_channels=channels)


# Capsule layer: 實作與你 TF 版本等價的動態路由，並使用 shared linear W (對應 add_weight)
class CapsuleLayer(nn.Module):
    def __init__(self, in_dim, num_capsules=10, capsule_dim=16, routing_iters=3):
        """
        in_dim: dimension of each input capsule (e.g., channels after CNN)
        num_capsules: number of output capsules
        capsule_dim: dimension of each output capsule
        """
        super().__init__()
        self.in_dim = in_dim
        self.num_capsules = num_capsules
        self.capsule_dim = capsule_dim
        self.routing_iters = routing_iters
        # W: (in_dim, num_capsules * capsule_dim)
        self.W = nn.Parameter(torch.empty(in_dim, num_capsules * capsule_dim))
        # Xavier / glorot uniform初始化，對應 TF 的 glorot_uniform
        nn.init.xavier_uniform_(self.W)

    def squash(self, s, dim=-1, eps=1e-9):
        squared_norm = (s ** 2).sum(dim=dim, keepdim=True)
        scale = squared_norm / (1.0 + squared_norm)
        return scale * s / torch.sqrt(squared_norm + eps)

    def forward(self, inputs):
        """
        inputs: (batch, input_len, in_dim)
        returns: (batch, num_capsules, capsule_dim)
        """
        # inputs_hat = einsum('bij,jk->bik', inputs, self.W)
        # inputs_hat shape -> (batch, input_len, num_capsules * capsule_dim)
        inputs_hat = torch.einsum('bij,jk->bik', inputs, self.W)
        batch_size = inputs_hat.size(0)
        input_len = inputs_hat.size(1)

        # reshape to (batch, input_len, num_capsules, capsule_dim)
        inputs_hat = inputs_hat.view(batch_size, input_len, self.num_capsules, self.capsule_dim)

        # initialize b zeros (batch, input_len, num_capsules)
        b = torch.zeros(batch_size, input_len, self.num_capsules, device=inputs.device, dtype=inputs.dtype)

        for i in range(self.routing_iters):
            c = torch.softmax(b, dim=2)  # (batch, input_len, num_capsules)
            # s = sum_i c_ij * inputs_hat_ij  -> (batch, num_capsules, capsule_dim)
            s = (c.unsqueeze(-1) * inputs_hat).sum(dim=1)
            v = self.squash(s)
            if i < self.routing_iters - 1:
                # agreement: sum over capsule_dim
                # inputs_hat * v.unsqueeze(1) -> (batch, input_len, num_capsules, capsule_dim)
                # reduce_sum over last dim -> (batch, input_len, num_capsules)
                agreement = (inputs_hat * v.unsqueeze(1)).sum(dim=-1)
                b = b + agreement
        return v  # (batch, num_capsules, capsule_dim)


# build_capsule_classifier: 接受 capsule 輸入 shape (time_steps, channels) 與超參數，回傳 nn.Module
def build_capsule_classifier(input_shape, num_capsules=10, capsule_dim=16, num_classes=4, routing_iters=3):
    """
    input_shape: tuple (time_steps, channels) representing CNN output per-sample (like Keras's dummy_output.shape[1:])
    returns: nn.Module which accepts input tensor shaped (batch, time_steps, channels)
    """
    class CapsuleClassifier(nn.Module):
        def __init__(self, in_time, in_channels, num_capsules, capsule_dim, num_classes, routing_iters):
            super().__init__()
            self.capsule_layer = CapsuleLayer(in_dim=in_channels,
                                              num_capsules=num_capsules,
                                              capsule_dim=capsule_dim,
                                              routing_iters=routing_iters)
            self.flatten = nn.Flatten()
            self.fc = nn.Linear(num_capsules * capsule_dim, num_classes)

        def forward(self, x):
            # x shape expected: (batch, time_steps, channels)
            # pass into capsule layer (which expects (batch, input_len, in_dim) ) -> (batch, num_capsules, capsule_dim)
            v = self.capsule_layer(x)
            flat = self.flatten(v)
            logits = self.fc(flat)
            probs = torch.sigmoid(logits)  # multi-label sigmoid as in your TF model
            return probs

    time_steps, channels = input_shape
    return CapsuleClassifier(time_steps, channels, num_capsules, capsule_dim, num_classes, routing_iters)


# margin_loss: 與你 TF 版本等價
def margin_loss(y_true, y_pred, m_plus=0.9, m_minus=0.1, lambd=0.5):
    """
    y_true: (batch, num_classes) float (0/1)
    y_pred: (batch, num_classes) float in [0,1] (sigmoid outputs)
    """
    y_true = y_true.float()
    y_pred = y_pred.float()
    term1 = y_true * (torch.clamp(m_plus - y_pred, min=0.0) ** 2)
    term2 = lambd * (1.0 - y_true) * (torch.clamp(y_pred - m_minus, min=0.0) ** 2)
    L = term1 + term2
    return torch.mean(torch.sum(L, dim=1))


In [4]:
# 設置默認工作目錄
folder_path = '/content/drive/MyDrive/20240715_大馬達收集資料'
os.chdir(folder_path)

# 定義資料夾和標籤（多標籤）
folders = ["齒輪不對中、軸承內斷", "齒輪不對中、軸承正常", "齒輪正常、軸承內斷", "齒輪正常、軸承正常", "齒輪磨耗、軸承內斷", "齒輪磨耗、軸承正常"]
labels_map = {
    # 都正常 - 0 齒輪磨耗 - 1  齒輪不對中 - 2  軸承內斷 - 3
    "齒輪正常、軸承正常": [0],
    "齒輪磨耗、軸承正常": [1],
    "齒輪不對中、軸承正常": [2],
    "齒輪正常、軸承內斷": [3],
    "齒輪磨耗、軸承內斷": [1, 3],
    "齒輪不對中、軸承內斷": [2, 3]
}

# 初始化數據和標籤
data = []
fault_labels = []

# 讀取每個資料夾中的CSV文件
for folder, labels in labels_map.items():
    for file in os.listdir(folder):
        if file.endswith(".csv"):
            filepath = os.path.join(folder, file)
            try:
                df = pd.read_csv(filepath, header=None)
                if df.shape[0] > 0 and df.shape[1] > 1:
                    data.append(df.iloc[0, 1:].values)
                    fault_labels.append(labels)
                else:
                    print(f"Skipping file {filepath}: Not enough data.")
            except Exception as e:
                print(f"Error reading file {filepath}: {e}")

# 確認是否有收集到數據
if not data:
    raise ValueError("No valid data found. Please check the CSV files.")

# 將數據轉換為 DataFrame
data_df = pd.DataFrame(data)

# 創建多標籤矩陣
num_classes = 4  # 0: 齒輪正常、軸承正常, 1: 齒輪磨耗, 2: 齒輪不對中, 3: 軸承內斷
multi_label_matrix = np.zeros((len(fault_labels), num_classes), dtype=int)

for i, label_list in enumerate(fault_labels):
    for label in label_list:
        multi_label_matrix[i, label] = 1

# 分割數據集
X_train, X_test, y_train, y_test = train_test_split(data_df, multi_label_matrix, test_size=0.3, random_state=42)

# 標準化數據
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 調整輸入數據的形狀
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# 打印數據集信息
print("訓練集大小:", X_train.shape, y_train.shape)
print("測試集大小:", X_test.shape, y_test.shape)

訓練集大小: (734, 12288, 1) (734, 4)
測試集大小: (315, 12288, 1) (315, 4)


In [5]:
from collections import Counter
print("--- 開始獨立計數程序 ---")

# --- 步驟 1: 從既有的標籤反向推導每筆資料的原始資料夾名稱 ---
# 建立一個從 "標籤組合" -> "資料夾名稱" 的反向對應字典
# 例如：(1, 3) -> "齒輪磨耗、軸承內斷"
# 我們使用 tuple 作為 key，因為 list 不能當作字典的 key
reverse_labels_map = {tuple(v): k for k, v in labels_map.items()}

# 根據 fault_labels 列表，建立一個長度與總樣本數相同的列表
# 其中每個元素都是該樣本對應的原始資料夾名稱
try:
    original_folders_reconstructed = [reverse_labels_map[tuple(lbl)] for lbl in fault_labels]
except KeyError as e:
    print(f"錯誤：在反向對應時找不到鍵 {e}。請檢查您的 `labels_map` 和 `fault_labels` 是否匹配。")
    # 這裡可以拋出錯誤或停止執行
    raise

# --- 步驟 2: 重新模擬 train_test_split 的 "索引" 分割過程 ---
# 建立一個從 0 到 N-1 的索引陣列 (N為總樣本數)
total_samples = len(data_df)
indices = np.arange(total_samples)

# 使用與您原始碼完全相同的參數 (test_size, random_state) 來分割索引
# 這能確保我們得到與您原始資料分割完全一致的索引結果
train_indices, test_indices = train_test_split(
    indices,
    test_size=0.3,
    random_state=42
)

# --- 步驟 3: 利用分割後的索引來計數 ---
# 根據 train_indices 從原始資料夾列表中取出所有訓練集樣本的來源
folders_in_train = [original_folders_reconstructed[i] for i in train_indices]
# 根據 test_indices 取出所有測試集樣本的來源
folders_in_test = [original_folders_reconstructed[i] for i in test_indices]

# 使用 Counter 直接計算各個來源資料夾在訓練集與測試集中出現的次數
train_counts = Counter(folders_in_train)
test_counts = Counter(folders_in_test)
original_counts = Counter(original_folders_reconstructed)

# --- 步驟 4: 格式化並印出結果 ---
print("\n----- 各資料夾樣本分佈 -----")
# 依照您 labels_map 的順序來印出，確保每次結果的順序都一樣
for folder in labels_map.keys():
    orig_count = original_counts.get(folder, 0)
    train_count = train_counts.get(folder, 0)
    test_count = test_counts.get(folder, 0)

    # 只有當該資料夾有數據時才印出
    if orig_count > 0:
        print(f"資料夾: {folder}")
        print(f"  原始樣本數: {orig_count}")
        print(f"  -> training: {train_count} 筆")
        print(f"  -> testing : {test_count} 筆")
        print("")

print("----- 總計檢查 -----")
print(f"原始總樣本數: {total_samples}")
print(f"訓練集合計 (應為 {len(X_train)}): {len(train_indices)}")
print(f"測試集合計 (應為 {len(X_test)}): {len(test_indices)}")
print(f"訓練 + 測試 合計: {len(train_indices) + len(test_indices)}")

--- 開始獨立計數程序 ---

----- 各資料夾樣本分佈 -----
資料夾: 齒輪正常、軸承正常
  原始樣本數: 171
  -> training: 118 筆
  -> testing : 53 筆

資料夾: 齒輪磨耗、軸承正常
  原始樣本數: 181
  -> training: 118 筆
  -> testing : 63 筆

資料夾: 齒輪不對中、軸承正常
  原始樣本數: 178
  -> training: 122 筆
  -> testing : 56 筆

資料夾: 齒輪正常、軸承內斷
  原始樣本數: 176
  -> training: 121 筆
  -> testing : 55 筆

資料夾: 齒輪磨耗、軸承內斷
  原始樣本數: 174
  -> training: 137 筆
  -> testing : 37 筆

資料夾: 齒輪不對中、軸承內斷
  原始樣本數: 169
  -> training: 118 筆
  -> testing : 51 筆

----- 總計檢查 -----
原始總樣本數: 1049
訓練集合計 (應為 734): 734
測試集合計 (應為 315): 315
訓練 + 測試 合計: 1049


In [10]:
# -------------------------
# PART 2: 模型建立、訓練與推論（直接可跑）
# -------------------------

# 假設以下變數已存在於環境中（來自你的原始資料）
# X_train, y_train, X_test, y_test  (numpy arrays)
# num_classes

# If X_train is 2D (N, T), make it (N, T, 1)
if X_train.ndim == 2:
    X_train = X_train[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

# input_shape as in your TF code: (timesteps, channels)
input_shape = (X_train.shape[1], X_train.shape[2])

# 1) 建 CNN (feature extractor)
cnn_model = build_feature_extractor(input_shape)

# 2) 用 dummy input 取得 CNN 輸出 shape (time_steps_after_cnn, channels_out)
# Prepare dummy (1, T, C) and forward through cnn_model to get shape
with torch.no_grad():
    dummy = torch.randn(1, input_shape[0], input_shape[1])
    dummy_out = cnn_model(dummy)  # shape (1, new_time, out_channels)
    capsule_input_shape = (dummy_out.shape[1], dummy_out.shape[2])  # (time_steps, channels)

# 3) 建 capsule classifier (接受 cnn 的輸出格式)
capsule_classifier = build_capsule_classifier(
    input_shape=capsule_input_shape,
    num_capsules=10,
    capsule_dim=16,
    num_classes=num_classes,
    routing_iters=3
)

# 4) 串接 CNN 和 capsule classifier 成一個完整模型
class DDCNNModel(nn.Module):
    def __init__(self, cnn, capsule_clf):
        super().__init__()
        self.cnn = cnn
        self.capsule_clf = capsule_clf

    def forward(self, x):
        # x expected (batch, timesteps, channels)
        y = self.cnn(x)  # (batch, new_time, channels_out)
        y = self.capsule_clf(y)  # (batch, num_classes) sigmoid probs
        return y

ddcnn_model = DDCNNModel(cnn_model, capsule_classifier)

# 5) training setup: optimizer, loss (use margin_loss), DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ddcnn_model.to(device)

batch_size = 32
epochs = 30
optimizer = torch.optim.Adam(ddcnn_model.parameters(), lr=1e-3)

# prepare datasets (y as float for margin_loss)
train_ds = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
test_ds = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32))
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

# 6) Training loop (使用 margin_loss)
for epoch in range(epochs):
    ddcnn_model.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        xb = xb.to(device)
        yb = yb.to(device)
        optimizer.zero_grad()
        y_pred = ddcnn_model(xb)  # (batch, num_classes), sigmoid probs
        loss = margin_loss(yb, y_pred)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * xb.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)

    # validation: 計算 exact-match accuracy (跟你最後的 success_count 一致)
    ddcnn_model.eval()
    all_preds = []
    all_trues = []
    with torch.no_grad():
        for xb, yb in test_loader:
            xb = xb.to(device)
            yb = yb.to(device)
            y_pred = ddcnn_model(xb)  # probs
            all_preds.append(y_pred.cpu().numpy())
            all_trues.append(yb.cpu().numpy())
    all_preds = np.vstack(all_preds)
    all_trues = np.vstack(all_trues)
    phi = 0.5
    pred_labels = (all_preds > phi).astype(int)
    success_count = np.sum(np.all(pred_labels == all_trues, axis=1))
    total_count = len(all_trues)
    val_acc = success_count / total_count
    print(f"Epoch {epoch+1}/{epochs} - train_loss: {epoch_loss:.6f} - val_exact_match_acc: {val_acc:.4%}")

# 7) 訓練完成後做 predict 與分析（等價於你的 Keras 部分）
ddcnn_model.eval()
with torch.no_grad():
    X_test_t = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_pred = ddcnn_model(X_test_t).cpu().numpy()  # (N_test, num_classes)
phi = 0.5
y_pred_labels = (y_pred > phi).astype(int)

# 計算總準確性（與你原本那段相同）
success_count = np.sum(np.all(y_pred_labels == y_test, axis=1))
total_count = len(y_test)
print(f"Number of successful predictions: {success_count} out of {total_count}")
print(f"Overall Accuracy: {success_count / total_count:.2%}")

# 多標籤混淆矩陣 (如你原本所做)
multi_confusion_matrix = multilabel_confusion_matrix(y_test, y_pred_labels)
label_names = ["正常(0)", "齒輪磨耗(1)", "齒輪不對中(2)", "軸承內斷(3)"] if num_classes==4 else [f"label_{i}" for i in range(num_classes)]
for i, cm in enumerate(multi_confusion_matrix):
    print(f"標籤 '{label_names[i]}' 的混淆矩陣:")
    print(cm)
    print("")


Epoch 1/30 - train_loss: 0.303881 - val_exact_match_acc: 11.7460%
Epoch 2/30 - train_loss: 0.270867 - val_exact_match_acc: 17.4603%
Epoch 3/30 - train_loss: 0.266496 - val_exact_match_acc: 7.3016%
Epoch 4/30 - train_loss: 0.265003 - val_exact_match_acc: 28.5714%
Epoch 5/30 - train_loss: 0.253730 - val_exact_match_acc: 28.8889%
Epoch 6/30 - train_loss: 0.230593 - val_exact_match_acc: 29.8413%
Epoch 7/30 - train_loss: 0.196576 - val_exact_match_acc: 28.8889%
Epoch 8/30 - train_loss: 0.184715 - val_exact_match_acc: 28.5714%
Epoch 9/30 - train_loss: 0.180440 - val_exact_match_acc: 32.6984%
Epoch 10/30 - train_loss: 0.180932 - val_exact_match_acc: 40.0000%
Epoch 11/30 - train_loss: 0.175703 - val_exact_match_acc: 18.0952%
Epoch 12/30 - train_loss: 0.177760 - val_exact_match_acc: 28.5714%
Epoch 13/30 - train_loss: 0.176311 - val_exact_match_acc: 28.8889%
Epoch 14/30 - train_loss: 0.165860 - val_exact_match_acc: 29.5238%
Epoch 15/30 - train_loss: 0.170043 - val_exact_match_acc: 47.6190%
Epoch

In [11]:
# --- 修正後的準確率計算邏輯 ---

total_count = len(y_test)
strict_success_count = 0      # 嚴格成功計數器
acceptable_success_count = 0  # 可接受的成功計數器

# 初始化用於計算各資料夾準確率的計數器
folder_test_count = Counter()
folder_correct_count = Counter() # 基於 "可接受" 標準的正確計數

# 遍歷每一筆測試資料
for i in range(total_count):
    y_true = y_test[i]
    y_pred = y_pred_labels[i]

    # 取得此筆測試資料的原始資料夾名稱並計數
    folder_name = folders_in_test[i]
    folder_test_count[folder_name] += 1

    is_acceptably_correct = False

    # 情況 1: 嚴格比對，預測與真實標籤完全相同
    if np.array_equal(y_true, y_pred):
        strict_success_count += 1
        is_acceptably_correct = True
    else:
        # 情況 2: 可接受的部份正確
        is_true_single_fault = (np.sum(y_true) == 1)
        is_pred_composite_fault = (np.sum(y_pred) > 1)

        if is_true_single_fault and is_pred_composite_fault:
            if np.array_equal(np.bitwise_and(y_true, y_pred), y_true):
                is_acceptably_correct = True

    # 如果這筆預測被判定為 "可接受的正確"，則更新整體與個別資料夾的計數器
    if is_acceptably_correct:
        acceptable_success_count += 1
        folder_correct_count[folder_name] += 1


# --- 打印兩種準確率結果 ---

print("----- 準確率評估報告 -----")

# 1. 嚴格準確率 (Strict Accuracy)
print(f"嚴格成功預測數量 (完全匹配): {strict_success_count} / {total_count}")
print(f"嚴格整體準確率 (Strict Accuracy): {strict_success_count / total_count:.2%}")
print("-" * 30)

# 2. 可接受準確率 (Acceptable Accuracy)
print(f"可接受的成功預測數量 (包含部分正確): {acceptable_success_count} / {total_count}")
print(f"可接受的整體準確率 (Acceptable Accuracy): {acceptable_success_count / total_count:.2%}")

# --- 新增：計算並輸出每個資料夾的 "可接受" 準確率 ---
print("\n----- 各資料夾可接受準確率 -----")
# 依您定義的順序遍歷，確保輸出順序一致
for folder in labels_map.keys():
    total = folder_test_count[folder]
    # 只有當該資料夾確實有測試樣本時才進行計算與打印
    if total > 0:
        correct = folder_correct_count[folder]
        accuracy = correct / total
        print(f"  {folder}: {total} test samples, {accuracy:.2%} accuracy")


# --- 多標籤混淆矩陣 (此部分不變) ---
print("\n\n----- 各標籤獨立混淆矩陣 -----")
multi_confusion_matrix = multilabel_confusion_matrix(y_test, y_pred_labels)

# 為了方便解讀，我們定義標籤名稱
label_names = ["正常(0)", "齒輪磨耗(1)", "齒輪不對中(2)", "軸承內斷(3)"]
for i, cm in enumerate(multi_confusion_matrix):
    print(f"標籤 '{label_names[i]}' 的混淆矩陣:")
    print(cm)
    print("") # 增加間隔

----- 準確率評估報告 -----
嚴格成功預測數量 (完全匹配): 270 / 315
嚴格整體準確率 (Strict Accuracy): 85.71%
------------------------------
可接受的成功預測數量 (包含部分正確): 302 / 315
可接受的整體準確率 (Acceptable Accuracy): 95.87%

----- 各資料夾可接受準確率 -----
  齒輪正常、軸承正常: 53 test samples, 100.00% accuracy
  齒輪磨耗、軸承正常: 63 test samples, 100.00% accuracy
  齒輪不對中、軸承正常: 56 test samples, 100.00% accuracy
  齒輪正常、軸承內斷: 55 test samples, 100.00% accuracy
  齒輪磨耗、軸承內斷: 37 test samples, 94.59% accuracy
  齒輪不對中、軸承內斷: 51 test samples, 78.43% accuracy


----- 各標籤獨立混淆矩陣 -----
標籤 '正常(0)' 的混淆矩陣:
[[262   0]
 [  0  53]]

標籤 '齒輪磨耗(1)' 的混淆矩陣:
[[187  28]
 [  2  98]]

標籤 '齒輪不對中(2)' 的混淆矩陣:
[[204   4]
 [ 11  96]]

標籤 '軸承內斷(3)' 的混淆矩陣:
[[172   0]
 [  0 143]]

